In [1]:
from dotenv import load_dotenv

_ = load_dotenv()
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "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"


from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain_core.utils.function_calling import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough, RunnableBranch
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_core.pydantic_v1 import BaseModel, Field
from sqlite3 import connect, ProgrammingError
from langchain.tools import tool
from email.message import EmailMessage
import ssl
import smtplib
import logging
import uuid

- Prompts Definition: Define two different prompt templates for replacing an existing address and adding a new address.
- LLM Instance: Create an instance of the language model (ChatOpenAI).
- Memory Buffer: Use ConversationBufferMemory to handle the chat history.
- LLM Chains: Create LLMChain objects for both prompts, integrating the memory buffer.
- Runnable Objects: Create RunnableLambda objects to wrap the chain runs.
- Parallel Execution: Use RunnableParallel to run both chains in parallel.
- Function Binding: Bind functions for address extraction.
- Agent Executor: Integrate everything into AgentExecutor, using the memory buffer to maintain context.

In [2]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('landlords.db')
cursor = conn.cursor()

# Create the table with an alphanumeric Contract_ID of at least 10 characters and an Email column
cursor.execute('''
CREATE TABLE IF NOT EXISTS address (
    Contract_ID TEXT,
    Date_of_Updation TEXT,
    Address TEXT,
    Email TEXT
)
''')

# Insert dummy data with alphanumeric Contract_ID (at least 10 characters) and unique emails for each Contract_ID
dummy_data = [
    ('A1B2C3D4E5', '2024-01-01', '123 Main St, Springfield', 'email1@example.com'),
    ('A1B2C3D4E5', '2024-02-01', '124 Main St, Springfield', 'email1@example.com'),
    ('F6G7H8I9J0', '2024-01-15', '456 Elm St, Shelbyville', 'email2@example.com'),
    ('F6G7H8I9J0', '2024-03-20', '457 Elm St, Shelbyville', 'email2@example.com'),
    ('K1L2M3N4O5', '2024-02-10', '789 Maple St, Capital City', 'email3@example.com'),
    ('P6Q7R8S9T0', '2024-04-05', '321 Oak St, Springfield', 'email4@example.com'),
    ('U1V2W3X4Y5', '2024-05-15', '654 Pine St, Shelbyville', 'email5@example.com'),
    ('Z1Y2X3W4V5', '2024-06-01', '987 Cedar St, Springfield', 'email6@example.com'),
    ('T6S5R4Q3P2', '2024-07-20', '741 Birch St, Shelbyville', 'email7@example.com'),
    ('N8M7L6K5J4', '2024-08-15', '852 Maple St, Capital City', 'email8@example.com'),
    ('I9H8G7F6E5', '2024-09-10', '963 Elm St, Springfield', 'email9@example.com'),
    ('D4C3B2A1Z0', '2024-10-05', '159 Oak St, Shelbyville', 'email10@example.com'),
    ('X1W2V3U4T5', '2024-11-15', '753 Pine St, Springfield', 'email11@example.com'),
    ('R5Q6P7O8N9', '2024-12-25', '246 Cedar St, Shelbyville', 'email12@example.com'),
    ('M3L2K1J0H9', '2025-01-30', '357 Birch St, Capital City', 'email13@example.com'),
    ('G8F7E6D5C4', '2025-02-14', '468 Maple St, Springfield', 'email14@example.com'),
    ('B2A1Z0Y9X8', '2025-03-10', '579 Elm St, Shelbyville', 'email15@example.com'),
    ('W3V4U5T6S7', '2025-04-05', '680 Oak St, Capital City', 'email16@example.com'),
    ('Q6P7O8N9M0', '2025-05-20', '791 Pine St, Springfield', 'email17@example.com')
]

# Insert data into the table
cursor.executemany('''
INSERT INTO address (Contract_ID, Date_of_Updation, Address, Email)
VALUES (?, ?, ?, ?)
''', dummy_data)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [3]:
# Function to query the table
def query_addresses(contract_id=None):
    # Connect to the SQLite database
    conn = sqlite3.connect('landlords.db')
    cursor = conn.cursor()

    if contract_id:
        # Query for a specific Contract_ID
        cursor.execute('SELECT * FROM address WHERE Contract_ID = ?', (contract_id,))
    else:
        # Query all records
        cursor.execute('SELECT * FROM address')
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Close the connection
    conn.close()
    
    return results

In [4]:
query_addresses(contract_id=None)

[('A1B2C3D4E5',
  '2024-01-01',
  '123 Main St, Springfield',
  'email1@example.com'),
 ('A1B2C3D4E5',
  '2024-02-01',
  '124 Main St, Springfield',
  'email1@example.com'),
 ('F6G7H8I9J0', '2024-01-15', '456 Elm St, Shelbyville', 'email2@example.com'),
 ('F6G7H8I9J0', '2024-03-20', '457 Elm St, Shelbyville', 'email2@example.com'),
 ('K1L2M3N4O5',
  '2024-02-10',
  '789 Maple St, Capital City',
  'email3@example.com'),
 ('P6Q7R8S9T0', '2024-04-05', '321 Oak St, Springfield', 'email4@example.com'),
 ('U1V2W3X4Y5',
  '2024-05-15',
  '654 Pine St, Shelbyville',
  'email5@example.com'),
 ('Z1Y2X3W4V5',
  '2024-06-01',
  '987 Cedar St, Springfield',
  'email6@example.com'),
 ('T6S5R4Q3P2',
  '2024-07-20',
  '741 Birch St, Shelbyville',
  'email7@example.com'),
 ('N8M7L6K5J4',
  '2024-08-15',
  '852 Maple St, Capital City',
  'email8@example.com'),
 ('I9H8G7F6E5', '2024-09-10', '963 Elm St, Springfield', 'email9@example.com'),
 ('D4C3B2A1Z0',
  '2024-10-05',
  '159 Oak St, Shelbyville',
  'e

In [5]:
output_parser = OpenAIFunctionsAgentOutputParser()

In [6]:
replace_address_prompt = ChatPromptTemplate.from_messages([
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with replacing an address from an existing list of addresses. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of replacing an address from an existing list of addresses.\n"
        "5. If the user asks about the conversation so far, answer within the context.\n"
        "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
        "7. If the user expresses confusion or asks for repetition, politely clarify and repeat the necessary information.\n"
        "8. If the user wants to go back one step, provide the previous stage's information.\n"
        "9. If the user selects an invalid option, politely ask them to select a valid option from the list.\n"
        "\n"
        "Example Conversations:\n"
        "\n"
        "Example 1:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: DEF4567\n"
        "Assistant: Fetching the list of addresses associated with contract ID DEF4567...\n"
        "1. 789 Pine St, Springfield, IL, 62706\n"
        "2. 101 Maple St, Springfield, IL, 62707\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID def56789 for further communications.\n"
        "\n"
        "Example 2:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: GHI8901\n"
        "Assistant: Fetching the list of addresses associated with contract ID GHI8901...\n"
        "1. 234 Elm St, Springfield, IL, 62708\n"
        "2. 345 Cedar St, Springfield, IL, 62709\n"
        "User: What time does the office close?\n"
        "Assistant: Please stay within the context of the conversation. Would you like to replace an address? Please select one address from the list.\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID ghi89101 for further communications.\n"
        "\n"
        "Example 3:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: JKL1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID JKL1234...\n"
        "1. 456 Birch St, Springfield, IL, 62710\n"
        "2. 567 Walnut St, Springfield, IL, 62711\n"
        "User: What did we talk about earlier?\n"
        "Assistant: We were discussing replacing your address. You provided contract ID JKL1234. Please select one address from the list.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID jkl12345 for further communications.\n"
        "\n"
        "Example 4:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: MNO5678\n"
        "Assistant: The provided contract ID does not match any records. Please check the contract ID and provide it again.\n"
        "User: MNO8765\n"
        "Assistant: Fetching the list of addresses associated with contract ID MNO8765...\n"
        "1. 678 Oak St, Springfield, IL, 62712\n"
        "2. 789 Ash St, Springfield, IL, 62713\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID mno87654 for further communications.\n"
        "\n"
        "Example 5:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: PQR9012\n"
        "Assistant: Fetching the list of addresses associated with contract ID PQR9012...\n"
        "1. 890 Fir St, Springfield, IL, 62714\n"
        "2. 901 Spruce St, Springfield, IL, 62715\n"
        "User: Can you confirm the details of address 1?\n"
        "Assistant: The address is 890 Fir St, Springfield, IL, 62714. Please confirm if you would like to replace with this address.\n"
        "User: Yes, replace with address 1.\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID pqr90123 for further communications.\n"
        "\n"
        "Example 6:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: STU3456 and the new address is 345 Pine St\n"
        "Assistant: Please provide only your contract ID to proceed with replacing an existing address.\n"
        "User: STU3456\n"
        "Assistant: Fetching the list of addresses associated with contract ID STU3456...\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID stu34567 for further communications.\n"
        "\n"
        "Example 7:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: VWX6789\n"
        "Assistant: Fetching the list of addresses associated with contract ID VWX6789...\n"
        "1. 234 Birch St, Springfield, IL, 62718\n"
        "2. 345 Cedar St, Springfield, IL, 62719\n"
        "User: Nevermind, I don't want to change it.\n"
        "Assistant: No problem. If you need further assistance, please let me know.\n"
        "\n"
        "Example 8:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: YZA1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID YZA1234...\n"
        "1. 567 Elm St, Springfield, IL, 62720\n"
        "2. 678 Pine St, Springfield, IL, 62721\n"
        "User: Are you sure these are the correct addresses?\n"
        "Assistant: Yes, these are the addresses associated with contract ID YZA1234. Please select one address to replace.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID yza12345 for further communications.\n"
        "\n"
        "Example 9:\n"
        "User: I don't understand.\n"
        "Assistant: I'm here to assist you. I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 10:\n"
        "User: Please repeat.\n"
        "Assistant: Certainly! I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 11:\n"
        "User: Can you tell me what was done so far?\n"
        "Assistant: Sure, we have discussed the process of replacing an address associated with a contract ID. The provided contract ID was contract_id. We fetched the list of addresses associated with this contract ID, and you selected an address to replace. If you have further questions, please let me know.\n"
        "\n"
        "Example 12:\n"
        "User: Can we go back one step?\n"
        "Assistant: Sure, let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the address you would like to replace by entering the corresponding number.\n"
        "\n"
        "Example 13:\n"
        "User: I forgot my contract ID.\n"
        "Assistant: No problem. Please check your records and provide your contract ID so we can proceed with replacing the address.\n"
        "If you cannot find your contract ID, please contact support for further assistance.\n"
        "\n"
        "Example 14:\n"
        "User: How do you know it's not correct?\n"
        "Assistant: The provided contract ID did not match any records in our system. Please double-check your contract ID and provide it again.\n"
        "\n"
        "Example 15:\n"
        "User: Yes\n"
        "Assistant: Great! Please provide your contract ID so I can assist you with replacing an address from the existing list of addresses.\n"
        "\n"
        "Example 16:\n"
        "User: I selected the wrong number for the address.\n"
        "Assistant: No problem. Let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the correct address by entering the corresponding number.\n"
        "\n"
        "Example 17:\n"
        "User: 3\n"
        "Assistant: The number you selected is not valid. Please select a valid address number from the list:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Enter the number corresponding to the address you want to replace."
    ),
     MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{user_input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])



class UpdateAddressInput(BaseModel):
    street: str = Field(description="Street address")
    city: str = Field(description="City")
    state: str = Field(description="State")
    zip_code: str = Field(description="Zip code")

@tool(args_schema=UpdateAddressInput)
def update_address(street: str, city: str, state: str, zip_code: str) -> str:
    """Update the user's address in the database."""
    # Logic to update address in the database
    return f"Address updated to {street}, {city}, {state}, {zip_code}"

In [7]:
update_email_address_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and polite landlord portal support agent. Your task is to assist users with updating their email addresses. Follow these guidelines strictly:\n"
               "1. If the user wants to update the email address, first prompt the user to provide the Contract ID.\n"
               "2. After receiving the Contract ID, prompt the user to provide the new email address.\n"
               "3. Use the provided Contract ID and email address to update the email in the database and send a confirmation email.\n"
               "4. If the user asks about the conversation so far, provide a summary based on the conversation history.\n"
               "5. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
               "6. If the user expresses confusion or asks for repetition, politely clarify and repeat the necessary information.\n"
               "7. If the user wants to go back one step, provide the previous stage's information.\n"
               "8. If the user selects an invalid option, politely ask them to select a valid option from the list.\n"
               "\n"
               "Example Conversations:\n"
               "\nExample 1 (Success):\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your Contract ID?\n"
               "User: It's A1B2C3D4E5.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: The new email is nithin.0897@gmail.com.\n"
               "Assistant: Thank you! I’m updating your email address now.\n"
               "Assistant: Email address updated successfully and confirmation email sent.\n"
               "Assistant: Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 2 (Contract ID Not Found):\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your Contract ID?\n"
               "User: It's INVALID123.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: The new email is nithin.0897@gmail.com.\n"
               "Assistant: Thank you! I’m updating your email address now.\n"
               "Assistant: I'm sorry, the provided Contract ID could not be found. Please check the ID and try again.\n"
               "Assistant: Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 3 (Email Update Failure):\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your Contract ID?\n"
               "User: It's A1B2C3D4E5.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: The new email is nithin.0897@gmail.com.\n"
               "Assistant: Thank you! I’m updating your email address now.\n"
               "Assistant: I'm sorry, there was an issue updating your email address. Please try again later or contact support for assistance.\n"
               "Assistant: Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"
               "\nExample 4 (Email Confirmation Failure):\n"
               "User: Hi, I need to update my email address.\n"
               "Assistant: Sure! I can help with that. Could you please provide your Contract ID?\n"
               "User: It's A1B2C3D4E5.\n"
               "Assistant: Thank you! Now, could you provide the new email address you’d like to use?\n"
               "User: The new email is nithin.0897@gmail.com.\n"
               "Assistant: Thank you! I’m updating your email address now.\n"
               "Assistant: Email address updated successfully but failed to send confirmation email: <error details>\n"
               "Assistant: Is there anything else I can assist you with?\n"
               "User: No, that's all. Thanks!\n"
               "Assistant: You're welcome! Have a great day!\n"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{user_input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [8]:
# Configure logging
logging.basicConfig(level=logging.INFO)

class EmailDetails(BaseModel):
    """Extracts Entities such as Contract ID and Email ID from the user input."""
    contract_id: str = Field(description="Contract ID")
    email_id: str = Field(description="Email ID")


def update_email_in_db(contract_id: str, email_id: str) -> str:
    """
    Update the email address for a given contract ID in the database.

    """
    try:
        conn = connect('landlords.db')
        cursor = conn.cursor()
        sql = "UPDATE address SET Email = ? WHERE Contract_ID = ?"
        cursor.execute(sql, (email_id, contract_id))
        if cursor.rowcount == 0:
            return "Contract ID not found."
        else:
            conn.commit()
            return "Email address updated successfully."
    except OperationalError as e:
        return f"Failed to update email address: {e}"
    finally:
        conn.close()

def send_confirmation_email(email_id: str) -> str:
    """
    Send a confirmation email to the new email address.
    
    """
    reference_id = str(uuid.uuid4())
    smtp_server = "smtp.gmail.com"
    smtp_port = 465
    smtp_username = "chatbot.testing.05@gmail.com"
    smtp_password = "dlwg ovip zkso dyow"

    subject = 'LLP- Address Update'
    body = f"""\
    We have received your request for updating the address for your contract ID. Kindly find your reference ID for further communications.

    Reference ID: {reference_id}

    Regards,
    Landlord portal
    """

    instance = EmailMessage()
    instance["From"] = smtp_username
    instance["To"] = email_id
    instance["Subject"] = subject
    instance.set_content(body)
    
    context = ssl.create_default_context()
    try:
        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as smtp:
            smtp.login(smtp_username, smtp_password)
            smtp.sendmail(smtp_username, email_id, instance.as_string())
            return "Email sent successfully."
    except Exception as e:
        return f"Failed to send email: {e}"
    
@tool(args_schema=EmailDetails)
def update_email_and_notify(contract_id: str, email_id: str) -> str:
    """
    Update the email address for a given contract ID in the database and send a confirmation email.

    This function performs the following steps:
    1. Updates the email address associated with the provided contract ID in the database.
    2. Sends a confirmation email to the new email address if the update is successful.

    Returns:
        str: A message indicating the result of the operation.
    """
    # Update email in the database
    update_result = update_email_in_db(contract_id, email_id)
    if "successfully" in update_result:
        # Send confirmation email if update was successful
        email_result = send_confirmation_email(email_id)
        if "successfully" in email_result:
            return f"{update_result} and {email_result}"
        else:
            return f"{update_result} but {email_result}"
    else:
        return update_result

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [19]:
classification_prompt = PromptTemplate.from_template(
 """classify the user input into either of the three classes only \n 1)inquiry\n 2)change email\n 3) update address \n"
               "If neither of the above three classes then classify it as others\n"
                "The output should adhere to the labels as mentioned above only\n"
                 "If you dont clearly understand, prompt the user to repeat again"),
 
   {user_input}
   
    """          
              
)


In [10]:
# Submit Inquiry Chain
inquiry_prompt = ChatPromptTemplate.from_messages([
    ("system","Please describe your inquiry in detail."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{user_input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")]
)

class SubmitInquiryInput(BaseModel):
    inquiry: str = Field(description="User inquiry")

@tool(args_schema=SubmitInquiryInput)
def submit_inquiry(inquiry: str) -> str:
    """Submit the user's inquiry."""
    # Logic to submit the inquiry
    return f"Inquiry submitted: {inquiry}"

In [11]:
general_prompt = ChatPromptTemplate.from_messages([
    ("system","Sorry, I didn't understand your request. Please try again."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{user_input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [12]:
output_parser = OpenAIFunctionsAgentOutputParser()

In [20]:
model = ChatOpenAI(temperature=0)
topic_chain = classification_prompt|model|StrOutputParser

submit_inquiry_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
)| inquiry_prompt | model | output_parser

general_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
)| general_prompt | model | output_parser

In [14]:
address_tools = [update_address]
address_update_functions = [format_tool_to_openai_function(f) for f in address_tools]
address_model = ChatOpenAI(temperature=0).bind(functions=address_update_functions)
update_address_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
)| replace_address_prompt | address_model | output_parser

address_memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
address_agent_executor = AgentExecutor(agent=update_address_chain,tools = address_tools,verbose=True, memory=address_memory)
# address_agent_executor.invoke({"question": 'what can you help me with?'})

/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [15]:
email_update_tools = [update_email_and_notify]
email_update_functions = [format_tool_to_openai_function(f) for f in email_update_tools]
email_update_model = ChatOpenAI(temperature=0).bind(functions=email_update_functions)

update_email_address_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
)| update_email_address_prompt | email_update_model | output_parser

email_update_memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
email_update_agent_executor_chain = AgentExecutor(agent=update_email_address_chain,tools = email_update_tools,verbose=True, memory=email_update_memory)

In [ ]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatAnthropic(model_name="claude-3-haiku-20240307")
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

In [25]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "update address" in x["topic"].lower(), address_agent_executor),
    (lambda x: "change email" in x["topic"].lower(), email_update_agent_executor_chain),
    (lambda x: "inquiry" in x['topic'].lower(), submit_inquiry_chain | submit_inquiry),
    general_chain,
)
full_chain = {"topic": topic_chain, "user_input": lambda x: x["user_input"]} |

In [26]:
full_chain.invoke({"user_input": " need to update email id "})

AttributeError: 'dict' object has no attribute 'invoke'

In [52]:
# Create the routing branch
branch = RunnableBranch(
    (lambda x: "update address" in x.return_values['output'].lower(),lambda x: address_agent_executor.invoke({"user_input": x}) ),
    (lambda x: "change email" in x.return_values['output'].lower(),lambda x: email_update_agent_executor_chain.invoke({"user_input": x}) ),
    (lambda x: "inquiry" in x.return_values['output'].lower(), submit_inquiry_chain | submit_inquiry),
    general_chain
)
# Full chain with topic classifier
full_chain = topic_chain | branch  

In [51]:
result = topic_chain.invoke({"user_input":'need to change the email?'})
result.return_values["output"]

INFO:httpx:HTTP Request: POST http://jupyter-api-proxy.internal.dlai/rev-proxy/langchain_langgraph/chat/completions "HTTP/1.1 200 OK"


'change email'

In [53]:
full_chain.invoke({"user_input":'need to change the email?'})

INFO:httpx:HTTP Request: POST http://jupyter-api-proxy.internal.dlai/rev-proxy/langchain_langgraph/chat/completions "HTTP/1.1 200 OK"


TypeError: 'AgentFinish' object is not subscriptable